# Episodic Memory를 사용한 디버깅 어시스턴트

## 개요

이 노트북은 reflection을 포함한 **AgentCore Episodic Memory**를 사용하여 지능형 **디버깅 어시스턴트**를 구축하는 방법을 보여줍니다. Agent는 과거 디버깅 세션에서 학습하고 과거 경험을 기반으로 컨텍스트 인식 가이드를 제공합니다.

### Episodic Memory란?

**Episodic Memory**는 구조화된 컨텍스트와 함께 완전한 상호작용 시퀀스(에피소드)를 캡처합니다. 고립된 사실을 저장하는 semantic memory와 달리, episodic memory는 다음을 보존합니다:
- **전체 대화 흐름**: 문제 진술부터 해결까지의 완전한 디버깅 세션
- **시간적 컨텍스트**: 수행된 작업의 순서와 타이밍
- **결과**: 디버깅 시도의 성공 또는 실패 여부
- **구조화된 턴**: 생각, 행동, 관찰이 포함된 개별 단계

![Episodic memory](./episodic_memory.png)

### Reflection이란?

**Reflection**은 여러 에피소드에서 자동으로 추출된 종합적인 인사이트입니다. 다음을 제공합니다:
- **패턴 인식**: 유사한 에피소드에서 공통 문제와 해결책
- **모범 사례**: 성공적인 디버깅 세션에서 잘 작동한 전략
- **일반적인 함정**: 실패한 시도를 기반으로 피해야 할 실수
- **전략적 가이드**: 유사한 문제에 접근하기 위한 높은 수준의 조언

**출력 구조:**
- **Episodes**: `debugging/{actorId}/sessions/{sessionId}`에 저장 - 전체 대화 추적
- **Reflections**: `debugging/{actorId}`에 저장 - 여러 에피소드에서 종합된 지식

### Episodic Memory를 언제 사용하나요?

다음과 같은 경우 episodic memory를 사용하세요:
1. **순차적 컨텍스트가 중요한 경우**: 작업의 순서와 결과가 중요한 경우 (예: 디버깅 워크플로우, 문제 해결 절차)
2. **경험으로부터 학습**: Agent가 과거의 성공과 실패를 분석하여 개선되기를 원하는 경우
3. **프로세스 검색**: 사용자가 "지난번에 X를 어떻게 해결했지?" 또는 "수행한 정확한 단계를 보여줘"를 회상해야 하는 경우

### 튜토리얼 세부 정보

| 정보 | 세부사항 |
|:------------|:--------|
| 튜토리얼 유형 | Reflections를 포함한 Episodic Memory |
| Agent 유형 | 디버깅 어시스턴트 |
| Framework | Strands Agents |
| LLM model | Claude Haiku 4.5 |
| Memory 전략 | Reflection Configuration을 포함한 Episodic Memory |
| 복잡도 | 중급 |

## 사전 요구사항

- Python 3.10+
- AgentCore Memory 권한이 있는 AWS 자격 증명
- AgentCore 서비스에 대한 액세스

## Step 1: 종속성 설치 및 설정

In [ ]:
%pip install -qr requirements.txt

In [ ]:
import json
import logging
import sys
import uuid
from datetime import datetime, timezone
from typing import List, Dict, Any
from pprint import pprint

# 로깅 설정
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("debugging-assistant")

# boto3: control plane과 data plane 작업용
import boto3

# Strands Agent framework import
from strands import Agent, tool

logger.info("✅ All dependencies imported successfully")

In [ ]:
import os
# 환경 설정
REGION = os.getenv('AWS_REGION', 'us-west-2')
# 세션 식별자
ACTOR_ID = "developer"

logger.info(f"Configuration set for region: {REGION}")
logger.info(f"Actor ID: {ACTOR_ID}")

## Step 2: Episodic 전략으로 Memory 생성

**Reflection Configuration**을 포함하는 **Episodic Memory Strategy**로 구성된 memory 리소스를 생성합니다. 이를 통해 다음이 가능합니다:
- 완전한 디버깅 세션 에피소드 저장
- 여러 에피소드에서 reflection 인사이트 자동 생성

In [ ]:
# control plane과 data plane 작업을 위한 boto3 클라이언트 초기화
client = boto3.client(
    'bedrock-agentcore',
    region_name=REGION,
)
memory_client = boto3.client(
  'bedrock-agentcore-control',
   region_name=REGION,
)

In [ ]:
# reflection을 포함한 episodic memory 전략을 dictionary로 정의
memory_name = "DebugAssistantEpisodic"

# Episodic memory는 customMemoryStrategy로 구현됨
episodic_strategy = {
    "episodicMemoryStrategy": {
      "name": "DebuggingEpisodeExtractor",
      "description": "Creates debugging session episodes with reflections per actor",
      "namespaces": [
        "debugging/{actorId}/sessions/{sessionId}/"
      ],
      "reflectionConfiguration": {
        "namespaces": [
          "debugging/{actorId}/" # episodic memory namespace의 정확한 prefix여야 함
        ]
      }
    }
}
logger.info(f"Strategy configured: {episodic_strategy['episodicMemoryStrategy']['name']}")
logger.info(f"Episode namespace: {episodic_strategy['episodicMemoryStrategy']['namespaces'][0]}")
logger.info(f"Reflection namespace: {episodic_strategy['episodicMemoryStrategy']['reflectionConfiguration']['namespaces'][0]}")

In [ ]:
# memory 가져오기 또는 생성
try:
    # 먼저 기존 memory 찾기 시도
    list_response = memory_client.list_memories(maxResults=100)
    memory_id = None
    for mem in list_response.get('memories', []):
        detail = memory_client.get_memory(memoryId=mem['id'])
        if detail['memory'].get('name') == memory_name:
            memory_id = mem['id']
            logger.info(f"✅ Using existing memory: {memory_id}")
            break
    
    # 찾지 못하면 생성
    if not memory_id:
        logger.info(f"Creating new memory: {memory_name}")
        response = memory_client.create_memory(
            name=memory_name,
            description="Episodic memory for debugging assistant with reflections",
            eventExpiryDuration=90,
            memoryStrategies=[episodic_strategy],
            clientToken=str(uuid.uuid4())
        )
        memory_id = response['memory']['id']
        logger.info(f"✅ Memory created: {memory_id}")
        
        # ACTIVE 상태 대기
        import time
        for _ in range(30):
            status = memory_client.get_memory(memoryId=memory_id)['memory']['status']
            if status == 'ACTIVE':
                logger.info("✅ Memory is ACTIVE")
                break
            time.sleep(10)
            
except Exception as e:
    logger.error(f"❌ Failed to get/create memory: {e}")
    raise

## Step 4: 과거 디버깅 세션으로 Memory 채우기

과거 디버깅 세션을 episodic memory에 로드해 보겠습니다. 각 세션은 완전한 디버깅 워크플로우를 나타냅니다.

In [ ]:
import os
import glob

# 모든 세션 데이터 파일 로드
data_dir = "./data"
session_files = sorted(glob.glob(f"{data_dir}/*.json"))

logger.info(f"Found {len(session_files)} session files to hydrate")

# 각 세션 hydrate
for session_file in session_files:
    session_name = os.path.basename(session_file).replace('.json', '')
    session_id = f"{session_name}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    logger.info(f"Hydrating session: {session_name}")
    
    # 대화 데이터 로드
    with open(session_file, 'r') as f:
        conversation = json.load(f)
    
    # payload 형식으로 변환
    payload = []
    for turn in conversation:
        conv_data = turn['conversational']
        payload.append({
            'conversational': {
                'content': {'text': conv_data['content']['text']},
                'role': conv_data['role']
            }
        })
    
    # boto3를 사용하여 직접 이벤트 생성
    event_timestamp = datetime.now(timezone.utc)
    result = client.create_event(
        memoryId=memory_id,
        actorId=ACTOR_ID,
        sessionId=session_id,
        eventTimestamp=event_timestamp,
        payload=payload
    )

    logger.info(f"   ✓ Stored {len(payload)} turns - Event ID: {result['event']['eventId']}")

logger.info(f"✅ Successfully hydrated {len(session_files)} debugging sessions")

In [ ]:
### memory record 목록 조회하여 LTM으로 추출되었는지 확인
import time
import pprint
reflection_namespace = f"debugging/{ACTOR_ID}/"
# time.sleep(60)
# boto3 클라이언트를 직접 사용하여 memory record 검색
response = client.list_memory_records(
    memoryId=memory_id,
    namespace=reflection_namespace,
    maxResults=20
)
memories = response.get('memoryRecordSummaries', [])
logger.info(f"   Found {len(memories)} memories")
if memories:
    pprint.pp(json.loads(memories[0]["content"]["text"]))

In [ ]:
# reflection과 episode가 생성되었는지 확인
import pprint
# boto3 클라이언트를 직접 사용하여 memory record 검색
response = client.retrieve_memory_records(
memoryId=memory_id,
namespace=f"debugging/{ACTOR_ID}/",
searchCriteria={
    'searchQuery': "memory leaks",
    "metadataFilters":[{"left": {"metadataKey": "x-amz-agentcore-memory-recordType"},
        "operator": "EQUALS_TO",
        "right": {"metadataValue": {"stringValue": "REFLECTION"}}
        }],          
    'topK': 10
},
    maxResults=20
)

reflections = response.get('memoryRecordSummaries', [])
logger.info(f"   Found {len(reflections)} relevant reflections")
if reflections:
    for reflection in reflections:
        reflection_json = json.loads(reflection['content']['text'])
        pprint.pp(reflection_json)

## Step 5: Memory 검색 Tool 생성

Agent를 위한 두 가지 특수 도구를 생성합니다:
1. **retrieve_process**: 상세한 단계별 프로세스를 위한 완전한 에피소드 추적 검색
2. **retrieve_reflection_knowledge**: 여러 에피소드에서 종합된 인사이트와 패턴 검색

In [ ]:
def count_tokens(text: str) -> int:
    """텍스트 문자열의 대략적인 토큰 수 계산."""
    return len(text)


def linearize_episodes(episodes: List[Dict], include_steps: bool = True,
                       include_reflections: bool = True) -> str:
    """에피소드 데이터를 사람이 읽을 수 있는 형식으로 선형화."""
    if not episodes:
        return "No relevant episodes found."

    output = []
    for idx, episode in enumerate(episodes, 1):
        content = episode.get('content', {})
        
        # text 필드에서 JSON 파싱
        if 'text' in content:
            try:
                episode_data = json.loads(content['text'])
            except json.JSONDecodeError:
                output.append(f"Episode {idx}: Unable to parse content\n")
                continue
        else:
            output.append(f"Episode {idx}: No content available\n")
            continue
        
        output.append(f"{'='*80}\nEpisode {idx}\n{'='*80}")
        output.append(f"**Situation:** {episode_data.get('situation', 'N/A')}")
        output.append(f"**Intent:** {episode_data.get('intent', 'N/A')}")
        output.append(f"**Assessment:** {episode_data.get('assessment', 'N/A')}\n")
        
        if include_steps:
            turns = episode_data.get('turns', [])
            if turns:
                output.append("**Debugging Steps:**")
                for turn_idx, turn in enumerate(turns, 1):
                    output.append(f"\nStep {turn_idx}:")
                    output.append(f"  Situation: {turn.get('situation', 'N/A')}")
                    output.append(f"  Action: {turn.get('action', 'N/A')}")
                    output.append(f"  Thought: {turn.get('thought', 'N/A')}")
        
        if include_reflections:
            reflection = episode_data.get('reflection')
            if reflection:
                output.append(f"\n**Reflection:** {reflection}\n")
    
    result = "\n".join(output)
    logger.info(f"   Episode tokens: {count_tokens(result)}")
    return result


def linearize_reflections(reflections: List[Dict]) -> str:
    """reflection 지식을 사람이 읽을 수 있는 형식으로 선형화."""
    if not reflections:
        return "No reflection knowledge found."

    output = []
    for idx, reflection in enumerate(reflections, 1):
        content = reflection.get('content', {})
        score = reflection.get('score', 0)
        
        # text 필드에서 JSON 파싱
        if 'text' in content:
            try:
                reflection_data = json.loads(content['text'])
            except json.JSONDecodeError:
                continue
        else:
            continue

        output.append(f"{'='*80}\nReflection {idx} (Relevance: {score:.2f})\n{'='*80}")
        output.append(f"**Title:** {reflection_data.get('title', 'Untitled')}")
        output.append(f"**Use Cases:** {reflection_data.get('use_cases', 'N/A')}")
        output.append(f"**Hints:** {reflection_data.get('hints', 'N/A')}\n")
    
    result = "\n".join(output)
    logger.info(f"   Reflection tokens: {count_tokens(result)}")
    return result


logger.info("✅ Linearization helper functions created")

In [ ]:
# agent를 위한 memory 검색 도구 생성

@tool
def retrieve_process(task: str, include_steps: bool = True) -> str:
    """
    주어진 작업을 해결하는 데 도움이 되는 예제 프로세스를 검색합니다. 구성 가능한 상세 수준으로
    완전한 디버깅 에피소드를 반환합니다.
    
    include_steps 매개변수를 사용하여 상세도 제어:
    - 사용자가 "정확한 단계", "전체 세부 정보", "어떻게 했는지", "어떤 단계를 수행했는지"를 요청하거나
      완전한 절차 정보가 필요한 경우 include_steps=True로 설정
    - 단계별 세부 정보 없이 높은 수준의 컨텍스트(상황, 의도, 평가)만으로 충분한
      패턴/모범 사례 쿼리의 경우 include_steps=False로 설정

    Args:
        task: 예제 프로세스가 필요한 해결할 작업
        include_steps: 상세한 단계별 턴을 포함할지 여부 (기본값: True)

    Returns:
        선택적 상세 단계가 포함된 형식화된 디버깅 에피소드
    """
    logger.info(f"🔍 Retrieving processes for task: {task} (include_steps={include_steps})")
    
    try:
        # 에피소드 namespace에서 검색
        namespace = f"debugging/{ACTOR_ID}/sessions/{session_id}/"
        
        # boto3 클라이언트를 직접 사용하여 memory record 검색
        response = client.retrieve_memory_records(
            memoryId=memory_id,
            namespace=namespace,
            searchCriteria={
                'searchQuery': task,
                'topK': 3
            },
            maxResults=20
        )
        
        episodes = response.get('memoryRecordSummaries', [])
        logger.info(f"   Found {len(episodes)} relevant episodes")
        
        # 구성 가능한 상세 수준으로 선형화
        return linearize_episodes(episodes, include_steps=include_steps, include_reflections=True)
        
    except Exception as e:
        logger.error(f"Error retrieving processes: {e}")
        return f"Error retrieving processes: {str(e)}"


@tool
def retrieve_reflection_knowledge(task: str, k: int = 5) -> str:
    """
    과거 agent 경험에서 종합된 reflection 지식을 검색합니다. 각 지식 항목에는
    (1) 설명적 제목, (2) 적용 시기에 대한 구체적인 사용 사례,
    (3) 성공적인 에피소드의 모범 사례와 실패한 에피소드에서 피해야 할 일반적인 함정을 포함한
    실행 가능한 힌트가 포함됩니다. 유사한 작업에 대한 전략적 가이드와 패턴을 얻는 데 사용합니다.

    Args:
        task: 전략적 가이드를 얻을 현재 작업
        k: 검색할 reflection 항목 수 (기본값: 5)

    Returns:
        과거 디버깅 경험에서 종합된 reflection 지식
    """
    logger.info(f"🔍 Retrieving reflection knowledge for task: {task}")
    
    try:
        # reflection namespace에서 검색
        namespace = f"debugging/{ACTOR_ID}/"
        
        # boto3 클라이언트를 직접 사용하여 memory record 검색
        response = client.retrieve_memory_records(
            memoryId=memory_id,
            namespace=namespace,
            searchCriteria={
                'searchQuery': "memory leaks",
                "metadataFilters":[{"left":{"metadataKey": "x-amz-agentcore-memory-recordType"},
                                    "operator": "EQUALS_TO",
                                    "right": {"metadataValue": {"stringValue": "REFLECTION"}}
                                    }],          
                'topK': k
            },
            maxResults=20
        )
        
        reflections = response.get('memoryRecordSummaries', [])
        logger.info(f"   Found {len(reflections)} relevant reflection insights")
        
        # reflection 선형화
        return linearize_reflections(reflections)
        
    except Exception as e:
        logger.error(f"Error retrieving reflections: {e}")
        return f"Error retrieving reflections: {str(e)}"


logger.info("✅ Memory retrieval tools created")

## Step 6: 디버깅 어시스턴트 Agent 생성

이제 memory 검색 도구가 장착된 Strands agent를 생성합니다.

In [ ]:
# 디버깅 어시스턴트 agent 생성
debugging_agent = Agent(
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[retrieve_process, retrieve_reflection_knowledge],
    system_prompt="""You are an expert Debugging Assistant with access to episodic memory.

Your capabilities:
- Retrieve past debugging episodes with complete step-by-step processes
- Access synthesized reflection knowledge showing patterns and best practices
- Provide guidance based on successful debugging experiences
- Warn about common pitfalls observed in past failures

When helping users:
1. Use retrieve_reflection_knowledge for strategic guidance, patterns, and high-level advice
2. Use retrieve_process when users need exact steps or want to recall what was done in a specific session
3. Synthesize insights from memory with your own reasoning
4. Be specific and actionable in your recommendations

Always explain your reasoning and cite relevant past experiences when available."""
)

logger.info("✅ Debugging assistant agent created")

## Step 7: 디버깅 어시스턴트 테스트

Agent가 episodic memory와 reflection을 어떻게 사용하는지 확인하기 위해 다양한 시나리오를 테스트해 보겠습니다.

### Test 1: 전략적 가이드 쿼리 (Reflection Knowledge)

In [ ]:
# Test 1: 메모리 이슈에 대한 전략적 가이드 얻기
query1 = "My application is running out of memory when processing large datasets. What should I look for?"

logger.info(f"\n{'='*80}")
logger.info(f"Test 1: Memory Issue Guidance")
logger.info(f"{'='*80}")
logger.info(f"Query: {query1}\n")

response1 = debugging_agent(query1)

print("\n" + "="*80)
print("AGENT RESPONSE:")
print("="*80)
print(response1)

### Test 2: 특정 프로세스 세부 정보 쿼리

In [ ]:
# Test 2: 특정 디버깅 프로세스 얻기
query2 = "Show me the exact steps for debugging a timeout issue with external API calls."

logger.info(f"\n{'='*80}")
logger.info(f"Test 2: API Timeout Debugging Process")
logger.info(f"{'='*80}")
logger.info(f"Query: {query2}\n")

response2 = debugging_agent(query2)

print("\n" + "="*80)
print("AGENT RESPONSE:")
print("="*80)
print(response2)

### Test 3: 패턴 인식 쿼리

In [ ]:
# Test 3: 동시성 이슈에 대한 패턴과 모범 사례 얻기
query3 = "What are common patterns and best practices for handling race conditions in multi-threaded applications?"

logger.info(f"\n{'='*80}")
logger.info(f"Test 3: Race Condition Patterns")
logger.info(f"{'='*80}")
logger.info(f"Query: {query3}\n")

response3 = debugging_agent(query3)

print("\n" + "="*80)
print("AGENT RESPONSE:")
print("="*80)
print(response3)

### Test 4: 특정 세션 회상

In [ ]:
# Test 4: 메모리 누수 세션에서 수행한 작업 회상
query4 = "What debugging steps did we take when we encountered the memory leak issue? I need the full details."

logger.info(f"\n{'='*80}")
logger.info(f"Test 4: Recall Memory Leak Session")
logger.info(f"{'='*80}")
logger.info(f"Query: {query4}\n")

response4 = debugging_agent(query4)

print("\n" + "="*80)
print("AGENT RESPONSE:")
print("="*80)
print(response4)

## Step 8: 직접 Memory 검사

episodic memory와 reflection에 저장된 내용을 직접 검사해 보겠습니다.

In [ ]:
# boto3를 사용하여 에피소드 직접 검사
logger.info("" + "="*80)
logger.info("Direct Episode Inspection")
logger.info("="*80)

# boto3를 직접 사용하여 에피소드 검색
namespace = f"debugging/{ACTOR_ID}/sessions/"
response = client.retrieve_memory_records(
    memoryId=memory_id,
    namespace=namespace,
    searchCriteria={
        'searchQuery': 'debugging',
        'topK': 2
    },
    maxResults=10
)

episodes = response.get('memoryRecordSummaries', [])

print(f"Found {len(episodes)} episodes in memory:")
for idx, episode in enumerate(episodes, 1):
    print(f"Episode {idx}:")
    pprint.pp(episode, depth=2, width=100)
    print("-" * 80)

In [ ]:
# boto3를 사용하여 reflection 직접 검사
import pprint
response = client.retrieve_memory_records(
memoryId=memory_id,
namespace=reflection_namespace,
searchCriteria={
    'searchQuery': "memory leaks",
    "metadataFilters":[{
        "left":{"metadataKey": "x-amz-agentcore-memory-recordType"},
        "operator": "EQUALS_TO",
        "right": {"metadataValue": {"stringValue": "REFLECTION"}}
                        }],          
    'topK': 10
},
    maxResults=20
)

reflections = response.get('memoryRecordSummaries', [])
logger.info(f"   Found {len(reflections)} relevant reflections")
if reflections:
    for reflection in reflections:
        reflection_json = json.loads(reflection['content']['text'])
        pprint.pp(reflection_json)

## 요약

### 달성한 내용

✅ boto3를 사용하여 reflection configuration으로 episodic memory 생성

✅ 과거 디버깅 세션으로 memory 채우기

✅ 에피소드와 reflection을 위한 특수 검색 도구 구축

✅ Strands framework를 사용하여 지능형 디버깅 어시스턴트 생성

✅ 전략적 가이드 검색 vs. 상세 프로세스 검색 시연

### 주요 요점

1. **Episodic Memory**는 시간적 컨텍스트와 함께 완전한 상호작용 시퀀스를 보존합니다
2. **Reflections**는 여러 에피소드에서 패턴과 인사이트를 자동으로 종합합니다
3. **선형화**는 LLM 소비를 위해 구조화된 데이터를 포맷하여 컨텍스트를 최적화합니다
4. **Tool 선택**이 중요합니다: 전략에는 reflection을, 상세 단계에는 에피소드를 사용하세요
5. **Boto3 직접 액세스**는 Genesis Memory API 작업에 대한 완전한 제어를 제공합니다

### 이 패턴을 언제 사용하나요

- 과거 티켓 해결에서 학습하는 **기술 지원 시스템**
- 성공적인 진단 절차를 회상하는 **문제 해결 어시스턴트**
- 지식 전달을 위해 전문가 워크플로우를 캡처하는 **교육 시스템**
- 과거 결과 분석이 더 나은 관행을 이끄는 **프로세스 개선** 시나리오

## 정리 (선택 사항)

완료 후 memory 리소스를 삭제하려면 주석을 해제하세요.

In [ ]:
# boto3를 사용하여 memory 리소스 삭제하려면 주석 해제
# try:
#     client.delete_memory(memoryId=memory_id, clientToken=str(uuid.uuid4()))
#     logger.info(f"✅ Successfully deleted memory: {memory_id}")
# except Exception as e:
#     logger.error(f"Error deleting memory: {e}")